
# Sistema de Planificación de Viajes

Alumna: Yvonne Echevarria.


In [39]:
# ============================================================================
# CELDA 1: INSTALACIÓN DE DEPENDENCIAS
# ============================================================================

print("📦 Instalando dependencias del sistema multi-agente...")
print("="*60)

# Dependencias principales
!pip install langgraph langchain langchain-google-genai wikipedia termcolor python-dotenv requests -q

print("\n✅ Instalación completada exitosamente")
print("="*60)


📦 Instalando dependencias del sistema multi-agente...

✅ Instalación completada exitosamente


In [44]:
# ============================================================================
# CELDA 2: CONFIGURACIÓN DE API KEY
# ============================================================================

import os
from google.colab import userdata

# Cargar API key desde secretos de Colab
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY

print(f"✅ API Key configurada: {GOOGLE_API_KEY[:2]}...{GOOGLE_API_KEY[-2:]}")

✅ API Key configurada: AI...ac


In [46]:
# ============================================================================
# CELDA 3: CONFIGURACIÓN DE LLM Y TOOLS
# ============================================================================

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
import wikipedia
import requests
from termcolor import colored

print(colored("🤖 Inicializando Google Gemini...", "cyan", attrs=["bold"]))

# ============================================================================
# CONFIGURACIÓN DEL MODELO
# ============================================================================

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-exp",
    google_api_key=os.environ['GOOGLE_API_KEY'],
    temperature=0.7,
    convert_system_message_to_human=True
)

print(colored("✅ Google Gemini 2.0 Flash inicializado", "green"))

# ============================================================================
# TOOL 1: Wikipedia API
# ============================================================================

def buscar_destino_wikipedia(destino: str) -> str:
    """
    Busca información sobre un destino turístico en Wikipedia.

    Args:
        destino: Nombre del destino (ciudad o país)

    Returns:
        str: Resumen de Wikipedia o mensaje de error
    """
    try:
        wikipedia.set_lang("es")
        resumen = wikipedia.summary(destino, sentences=10)
        return f"📍 Información sobre {destino}:\n\n{resumen}"

    except wikipedia.exceptions.DisambiguationError as e:
        print(colored(f"⚠️  Múltiples resultados, usando: {e.options[0]}", "yellow"))
        resumen = wikipedia.summary(e.options[0], sentences=10)
        return f"📍 Información sobre {e.options[0]}:\n\n{resumen}"

    except wikipedia.exceptions.PageError:
        return f"❌ No se encontró información sobre '{destino}' en Wikipedia."

    except Exception as e:
        return f"❌ Error al buscar información: {str(e)}"

# ============================================================================
# TOOL 2: API de Clima (Open-Meteo)
# ============================================================================

@tool
def obtener_clima_destino(ciudad: str) -> dict:
    """
    Obtiene información del clima actual de una ciudad usando Open-Meteo API.

    Args:
        ciudad: Nombre de la ciudad para consultar el clima

    Returns:
        dict: Información del clima (temperatura, descripción, humedad, recomendación)
    """
    try:
        # Paso 1: Obtener coordenadas de la ciudad
        geocoding_url = f"https://geocoding-api.open-meteo.com/v1/search?name={ciudad}&count=1&language=es&format=json"
        geo_response = requests.get(geocoding_url, timeout=5)
        geo_data = geo_response.json()

        if not geo_data.get("results"):
            return {
                "ciudad": ciudad,
                "error": f"No se encontró la ciudad: {ciudad}",
                "temperatura": "N/A",
                "descripcion": "Ciudad no encontrada"
            }

        # Obtener coordenadas
        lat = geo_data["results"][0]["latitude"]
        lon = geo_data["results"][0]["longitude"]
        nombre_completo = geo_data["results"][0]["name"]

        # Paso 2: Obtener clima actual
        weather_url = f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current=temperature_2m,relative_humidity_2m,weather_code&timezone=auto"
        weather_response = requests.get(weather_url, timeout=5)
        weather_data = weather_response.json()

        current = weather_data["current"]

        # Mapear códigos de clima a descripciones en español
        weather_codes = {
            0: "Despejado ☀️",
            1: "Mayormente despejado 🌤️",
            2: "Parcialmente nublado ⛅",
            3: "Nublado ☁️",
            45: "Neblina 🌫️",
            48: "Niebla densa 🌫️",
            51: "Llovizna ligera 🌦️",
            61: "Lluvia ligera 🌧️",
            63: "Lluvia moderada 🌧️",
            65: "Lluvia intensa ⛈️",
            71: "Nevada ligera 🌨️",
            73: "Nevada moderada ❄️",
            75: "Nevada intensa ❄️",
            95: "Tormenta eléctrica ⛈️"
        }

        weather_desc = weather_codes.get(current["weather_code"], "Condiciones variables 🌤️")

        # Generar recomendación
        weather_code = current["weather_code"]
        if weather_code in [51, 61, 63, 65, 95]:
            recomendacion = "Lleva paraguas ☂️"
        elif weather_code in [71, 73, 75]:
            recomendacion = "Lleva abrigo y ropa térmica 🧥"
        elif weather_code == 0:
            recomendacion = "Excelente clima para visitar 😊"
        else:
            recomendacion = "Buen clima en general ✅"

        return {
            "ciudad": nombre_completo,
            "temperatura": f"{current['temperature_2m']}°C",
            "descripcion": weather_desc,
            "humedad": f"{current['relative_humidity_2m']}%",
            "recomendacion": recomendacion
        }

    except Exception as e:
        return {
            "ciudad": ciudad,
            "error": f"No se pudo obtener el clima: {str(e)}",
            "temperatura": "N/A",
            "descripcion": "Información no disponible",
            "recomendacion": "Consulta el clima localmente"
        }

# ============================================================================
# LISTA DE TOOLS DISPONIBLES
# ============================================================================

tools = [obtener_clima_destino]

print(colored("\n✅ Tools configuradas:", "green", attrs=["bold"]))
print(colored("   1. 📚 Wikipedia API - Información de destinos", "cyan"))
print(colored("   2. 🌤️  Open-Meteo API - Clima en tiempo real", "cyan"))

# ============================================================================
# PRUEBA DE FUNCIONAMIENTO
# ============================================================================

print(colored("\n🧪 Probando sistema...", "yellow"))

# Test LLM
try:
    test_response = llm.invoke("Di solo 'OK'")
    print(colored("✅ LLM funcionando", "green"))
except Exception as e:
    print(colored(f"❌ Error en LLM: {e}", "red"))

# Test Wikipedia
try:
    test_wiki = buscar_destino_wikipedia("París")
    print(colored("✅ Wikipedia funcionando", "green"))
except Exception as e:
    print(colored(f"❌ Error en Wikipedia: {e}", "red"))

# Test Clima
try:
    test_clima = obtener_clima_destino.invoke({"ciudad": "París"})
    print(colored(f"✅ API Clima funcionando: {test_clima.get('temperatura', 'N/A')}", "green"))
except Exception as e:
    print(colored(f"❌ Error en Clima: {e}", "red"))

print(colored("\n🎉 Sistema configurado y listo", "green", attrs=["bold"]))


🤖 Inicializando Google Gemini...
✅ Google Gemini 2.0 Flash inicializado

✅ Tools configuradas:
   1. 📚 Wikipedia API - Información de destinos
   2. 🌤️  Open-Meteo API - Clima en tiempo real

🧪 Probando sistema...
✅ LLM funcionando
✅ Wikipedia funcionando
✅ API Clima funcionando: 10.4°C

🎉 Sistema configurado y listo


In [14]:
# CELDA 4: Definición del Estado Compartido
from typing import TypedDict, Annotated, Sequence
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages

class TravelState(TypedDict):
    """
    Estado compartido entre todos los agentes del sistema.

    Este estado viaja por todo el grafo y cada agente puede:
    - Leer información
    - Agregar/modificar información
    - Acumular mensajes conversacionales
    """

    # ============ CONVERSACIÓN ============
    messages: Annotated[Sequence[BaseMessage], add_messages]
    # Historial completo de la conversación usuario-sistema
    # add_messages es una función de REDUCCIÓN que concatena mensajes

    # ============ INFORMACIÓN DEL VIAJE ============
    destination: str  # Ciudad/país de destino (ej: "París")
    duration: str     # Duración del viaje (ej: "5 días")
    activities: str   # Tipo de actividades (ej: "museos y gastronomía")
    budget: str       # Presupuesto aproximado (ej: "medio")

    # ============ DECISIONES DE FLUJO ============
    supervisor_decision: str  # "CONTINUE" o "INVESTIGATE"
    interview_complete: bool  # ¿Tenemos toda la info del usuario?

    # ============ RESULTADOS ============
    destination_research: str  # Info de Wikipedia sobre el destino
    travel_itinerary: str      # Itinerario final día por día


print(colored("✅ Estado compartido definido", "green"))
print(colored("📊 Variables del estado:", "cyan"))
print(colored("   - messages: Historial conversacional", "white"))
print(colored("   - destination: Destino del viaje", "white"))
print(colored("   - duration: Duración del viaje", "white"))
print(colored("   - activities: Tipo de actividades", "white"))
print(colored("   - budget: Presupuesto", "white"))
print(colored("   - supervisor_decision: Control de flujo", "white"))
print(colored("   - interview_complete: Flag de completitud", "white"))
print(colored("   - destination_research: Info del destino", "white"))
print(colored("   - travel_itinerary: Itinerario final", "white"))

✅ Estado compartido definido
📊 Variables del estado:
   - messages: Historial conversacional
   - destination: Destino del viaje
   - duration: Duración del viaje
   - activities: Tipo de actividades
   - budget: Presupuesto
   - supervisor_decision: Control de flujo
   - interview_complete: Flag de completitud
   - destination_research: Info del destino
   - travel_itinerary: Itinerario final


In [15]:
# CELDA 5: Prompts Especializados
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# ============================================================
# PROMPT 1: ENTREVISTADOR DE VIAJES
# ============================================================
interviewer_prompt = ChatPromptTemplate.from_messages([
    ("system", """
Eres un agente de viajes experto, amigable y entusiasta.

TU MISIÓN:
1. Descubrir QUÉ quiere el viajero (destino, actividades, estilo de viaje)
2. Entender CUÁNTO tiempo tiene (duración del viaje)
3. Conocer sus PREFERENCIAS (tipo de actividades)
4. Evaluar su PRESUPUESTO aproximado

CÓMO DEBES ACTUAR:
- Haz UNA pregunta a la vez (no abrumes)
- Sé conversacional y natural
- Muestra entusiasmo por los destinos
- Da opciones cuando sea útil

INFORMACIÓN QUE NECESITAS RECOPILAR:
✈️ Destino: ¿A dónde quiere viajar?
📅 Duración: ¿Cuántos días?
🎯 Actividades: ¿Qué le gusta? (cultura, aventura, playa, gastronomía, etc.)
💰 Presupuesto: ¿Aproximado? (bajo, medio, alto)

IMPORTANTE: Cuando tengas TODA esta información clara, termina tu mensaje diciendo EXACTAMENTE:
"ENTREVISTA_COMPLETA: Tengo todo lo necesario para planificar tu viaje."

Si falta alguna información, sigue preguntando de forma natural.
"""),
    MessagesPlaceholder(variable_name="messages")
])

# ============================================================
# PROMPT 2: SUPERVISOR
# ============================================================
supervisor_prompt = ChatPromptTemplate.from_messages([
    ("system", """
Eres un supervisor que detecta cuando el entrevistador tiene toda la información necesaria.

Analiza SOLO el último mensaje del entrevistador.

REGLAS SIMPLES:
- Si el mensaje contiene "ENTREVISTA_COMPLETA:" → responde exactamente: "INVESTIGATE"
- Si no contiene esa frase → responde exactamente: "CONTINUE"

Responde SOLO con una palabra: "INVESTIGATE" o "CONTINUE"
Sin puntos, sin explicaciones, solo la palabra.
"""),
    MessagesPlaceholder(variable_name="messages")
])

# ============================================================
# PROMPT 3: INVESTIGADOR DE DESTINOS
# ============================================================
wikipedia_prompt = ChatPromptTemplate.from_messages([
    ("system", """
Eres un investigador de destinos turísticos experto.

Recibirás información de Wikipedia sobre un destino.

TU TRABAJO:
1. Extraer los datos MÁS RELEVANTES para un viajero:
   - Qué ver (atracciones principales)
   - Cultura y costumbres importantes
   - Clima típico
   - Tips de viaje

2. Organizar la información de forma clara y útil

3. Enfocarte en lo PRÁCTICO, no en historia extensa

Devuelve un resumen estructurado y fácil de leer.
"""),
    MessagesPlaceholder(variable_name="messages")
])

# ============================================================
# PROMPT 4: GENERADOR DE ITINERARIO
# ============================================================
study_guide_prompt = ChatPromptTemplate.from_messages([
    ("system", """
Eres un planificador de viajes profesional y creativo.

Vas a crear un ITINERARIO DÍA POR DÍA personalizado.

INFORMACIÓN QUE TENDRÁS:
- Destino del viaje
- Duración (número de días)
- Preferencias de actividades
- Presupuesto
- Información del destino (de Wikipedia)

CÓMO CREAR EL ITINERARIO:

1. ESTRUCTURA:
   DÍA 1: [Título del día]
   - Mañana: [Actividades]
   - Tarde: [Actividades]
   - Noche: [Actividades]

   DÍA 2: [Título del día]
   ...

2. PERSONALIZACIÓN:
   - Adapta las actividades a las preferencias del usuario
   - Respeta el presupuesto (bajo = actividades gratuitas, medio = mix, alto = premium)
   - Incluye variedad: no solo museos, no solo restaurantes

3. DETALLES ÚTILES:
   - Nombres de lugares específicos
   - Tips prácticos (mejor hora para visitar, cómo llegar)
   - Alternativas por si llueve

4. FORMATO:
   - Usa emojis para hacerlo visual
   - Sé específico pero conciso
   - Incluye estimado de tiempo

Crea un itinerario que el viajero pueda seguir fácilmente.
"""),
    MessagesPlaceholder(variable_name="messages")
])

print(colored("✅ Prompts especializados definidos", "green"))
print(colored("🎭 4 Agentes configurados:", "cyan"))
print(colored("   1. 👤 Entrevistador de Viajes", "white"))
print(colored("   2. 🔍 Supervisor de Completitud", "white"))
print(colored("   3. 🗺️  Investigador de Destinos", "white"))
print(colored("   4. 📋 Generador de Itinerario", "white"))

✅ Prompts especializados definidos
🎭 4 Agentes configurados:
   1. 👤 Entrevistador de Viajes
   2. 🔍 Supervisor de Completitud
   3. 🗺️  Investigador de Destinos
   4. 📋 Generador de Itinerario


In [51]:
# ============================================================================
# CELDA 6: IMPLEMENTACIÓN DE LOS NODOS DEL SISTEMA
# ============================================================================

from langchain_core.messages import HumanMessage, AIMessage

# ============================================================================
# NODO 1: ENTREVISTADOR
# ============================================================================

async def interviewer_node(state: TravelState):
    """
    Nodo 1: Entrevistador de viajes

    Responsabilidades:
    - Hacer preguntas al usuario sobre su viaje
    - Recopilar: destino, duración, actividades, presupuesto
    - Una pregunta a la vez, conversacional
    """
    print(colored("\n👤 [ENTREVISTADOR ACTIVADO]", "magenta", attrs=["bold"]))

    messages = state["messages"]

    # Si no hay mensajes, generar saludo inicial (sin llamar al LLM)
    if len(messages) == 0:
        initial_message = AIMessage(content="¡Hola! 👋 Soy tu asistente personal de viajes. ¿A qué destino te gustaría viajar? 🌍✈️")
        return {"messages": [initial_message]}

    # Prompt del entrevistador
    interviewer_prompt = f"""
Eres un agente de viajes experto, amigable y entusiasta.

TU MISIÓN:
Recopilar información para planificar el viaje perfecto.

INFORMACIÓN NECESARIA:
1. ✈️ Destino: ¿A dónde quiere viajar?
2. 📅 Duración: ¿Cuántos días?
3. 🎯 Actividades: ¿Qué le gusta hacer? (cultura, aventura, playa, gastronomía, etc.)
4. 💰 Presupuesto: ¿Aproximado? (bajo/medio/alto)

REGLAS:
- Haz UNA pregunta a la vez
- Sé conversacional y natural
- Muestra entusiasmo por los destinos
- Cuando tengas las 4 respuestas, di: "ENTREVISTA_COMPLETA: Tengo todo lo necesario para planificar tu viaje."

Conversación hasta ahora:
{chr(10).join([f"{'Usuario' if isinstance(msg, HumanMessage) else 'Asistente'}: {msg.content}" for msg in messages])}

Tu siguiente pregunta:
"""

    response = await llm.ainvoke(interviewer_prompt)

    return {"messages": [response]}

# ============================================================================
# NODO 2: SUPERVISOR (VERSIÓN MEJORADA - MÁS AGRESIVO)
# ============================================================================

async def supervisor_node(state: TravelState):
    """
    Nodo 2: Supervisor que determina si hay suficiente información

    Responsabilidades:
    - Evaluar si se recopiló toda la información necesaria
    - Decidir: CONTINUE (seguir preguntando) o INVESTIGATE (avanzar)
    - Sistema de respaldo más agresivo
    """
    print(colored("\n🔍 [SUPERVISOR ACTIVADO]", "blue", attrs=["bold"]))

    messages = state["messages"]

    # Revisar último mensaje del entrevistador
    last_message = messages[-1].content if messages else ""

    # DETECCIÓN PRIMARIA: Buscar señales de finalización
    completion_signals = [
        "entrevista_completa",
        "tengo todo",
        "listo para planificar",
        "toda la información",
        "suficiente información",
        "perfecto",
        "excelente"
    ]

    if any(signal in last_message.lower() for signal in completion_signals):
        print(colored("   ✅ Entrevista completa detectada", "green"))
        print(colored("   ℹ️  Detectado por señal del LLM", "cyan"))
        return {
            "supervisor_decision": "INVESTIGATE",
            "interview_complete": True
        }

    # SISTEMA DE RESPALDO AGRESIVO: Verificación determinística
    full_conversation = " ".join([msg.content.lower() for msg in messages])

    # Verificar que se mencionaron los 4 temas clave
    has_destination = any(word in full_conversation for word in [
        "viajar", "ir a", "visitar", "destino", "ciudad", "país",
        "tokio", "parís", "londres", "roma", "barcelona", "nueva york",
        "méxico", "perú", "argentina", "chile", "colombia", "españa",
        "brasil", "canadá", "italia", "francia", "alemania", "japón",
        "china", "tailandia", "grecia", "egipto", "dubai", "miami"
    ])

    has_duration = any(word in full_conversation for word in [
        "días", "día", "semanas", "semana", "mes", "meses", "fin de semana"
    ]) or any(str(i) in full_conversation for i in range(1, 31))

    has_activities = any(word in full_conversation for word in [
        "cultura", "museo", "templo", "playa", "aventura", "gastronomía",
        "comida", "restaurante", "historia", "naturaleza", "deportes",
        "compras", "vida nocturna", "relajar", "fiesta", "turismo",
        "monumentos", "arte", "fotografía", "senderismo", "buceo"
    ])

    has_budget = any(word in full_conversation for word in [
        "bajo", "medio", "alto", "económico", "presupuesto", "barato",
        "caro", "moderado", "dólares", "euros", "soles", "pesos",
        "dinero", "gastar", "precio"
    ])

    # Contar respuestas del usuario
    user_responses = len([msg for msg in messages if isinstance(msg, HumanMessage)])

    print(colored(f"   ⏳ Evaluando entrevista...", "yellow"))
    print(colored(f"   📊 Información detectada:", "yellow"))
    print(colored(f"      • Destino: {'✅' if has_destination else '❌'}", "green" if has_destination else "red"))
    print(colored(f"      • Duración: {'✅' if has_duration else '❌'}", "green" if has_duration else "red"))
    print(colored(f"      • Actividades: {'✅' if has_activities else '❌'}", "green" if has_activities else "red"))
    print(colored(f"      • Presupuesto: {'✅' if has_budget else '❌'}", "green" if has_budget else "red"))
    print(colored(f"   👤 Respuestas del usuario: {user_responses}", "yellow"))

    # CRITERIO AGRESIVO: Si tenemos 3 de 4 temas Y mínimo 3 turnos → completar
    topics_completed = sum([has_destination, has_duration, has_activities, has_budget])

    if topics_completed >= 3 and user_responses >= 3:
        print(colored("   ✅ Entrevista completa detectada", "green", attrs=["bold"]))
        print(colored(f"   ℹ️  Detectado por respaldo ({topics_completed}/4 temas, {user_responses} respuestas)", "cyan"))
        return {
            "supervisor_decision": "INVESTIGATE",
            "interview_complete": True
        }

    # Si no está completo, continuar
    print(colored(f"   ⏳ Continuando entrevista... (faltan {4-topics_completed} temas)", "yellow"))
    return {
        "supervisor_decision": "CONTINUE",
        "interview_complete": False
    }

# ============================================================================
# NODO 3: INVESTIGADOR DE DESTINOS (CON CLIMA)
# ============================================================================

async def wikipedia_node(state: TravelState):
    """
    Nodo 3: Investiga el destino usando Wikipedia + Clima en tiempo real

    Responsabilidades:
    1. Extrae información estructurada de la conversación
    2. Busca información del destino en Wikipedia
    3. Consulta el clima actual del destino (NUEVA FUNCIONALIDAD)
    4. Procesa y estructura toda la información
    """
    print(colored("\n🗺️  [INVESTIGADOR DE DESTINOS ACTIVADO]", "cyan", attrs=["bold"]))

    messages = state["messages"]

    # ========================================================================
    # PASO 1: Construir conversación completa
    # ========================================================================
    full_conversation = "\n".join([
        f"{'Usuario' if isinstance(msg, HumanMessage) else 'Asistente'}: {msg.content}"
        for msg in messages
    ])

    print(colored(f"   📝 Conversación completa capturada ({len(messages)} mensajes)", "yellow"))

    # ========================================================================
    # PASO 2: Extraer información estructurada usando LLM
    # ========================================================================
    extraction_prompt = f"""
Analiza esta conversación sobre planificación de viaje y extrae la información clave.

CONVERSACIÓN:
{full_conversation}

EXTRAE la siguiente información en este formato EXACTO:
DESTINO: [ciudad o país mencionado]
DURACIÓN: [número de días mencionados]
ACTIVIDADES: [tipos de actividades mencionadas]
PRESUPUESTO: [bajo, medio o alto]

Si algo no está claro, indica [no especificado].
"""

    extraction_response = await llm.ainvoke(extraction_prompt)
    extracted_info = extraction_response.content

    print(colored("   📊 Información extraída:", "yellow"))
    print(colored(f"   {extracted_info}", "white"))

    # ========================================================================
    # PASO 3: Identificar el destino
    # ========================================================================
    destination = "no especificado"
    for line in extracted_info.split('\n'):
        if line.startswith('DESTINO:'):
            destination = line.replace('DESTINO:', '').strip()
            break

    print(colored(f"   📍 Destino identificado: {destination}", "yellow"))

    # ========================================================================
    # PASO 4: Buscar información en Wikipedia
    # ========================================================================
    if destination and destination != "no especificado" and destination != "[no especificado]":
        print(colored("   🔎 Buscando información en Wikipedia...", "yellow"))
        wikipedia_content = buscar_destino_wikipedia(destination)

        # ====================================================================
        # PASO 5: Obtener clima actual (NUEVA FUNCIONALIDAD) 🌤️
        # ====================================================================
        print(colored("   🌤️  Consultando clima actual del destino...", "yellow"))
        clima_info = obtener_clima_destino.invoke({"ciudad": destination})

        # Construir sección de clima
        if "error" not in clima_info:
            clima_texto = f"""
🌤️ CLIMA ACTUAL EN {clima_info['ciudad']}:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
🌡️ Temperatura: {clima_info['temperatura']}
☁️ Condiciones: {clima_info['descripcion']}
💧 Humedad: {clima_info['humedad']}
💡 Recomendación: {clima_info['recomendacion']}
"""
        else:
            clima_texto = f"\n🌤️ CLIMA: No se pudo obtener información del clima para {destination}\n"

    else:
        wikipedia_content = f"No se pudo identificar un destino específico en la conversación."
        clima_texto = ""

    # ========================================================================
    # PASO 6: Procesar información con LLM
    # ========================================================================
    print(colored("   ⚙️  Procesando información...", "yellow"))

    processing_prompt = f"""
Eres un experto en turismo. Procesa esta información y crea una guía práctica.

INFORMACIÓN DE WIKIPEDIA:
{wikipedia_content}

{clima_texto}

CONTEXTO DEL VIAJE:
{extracted_info}

INSTRUCCIONES:
Crea una guía turística concisa con:
1. ¿Qué ver? (principales atracciones)
2. Cultura y costumbres importantes
3. Información del clima actual y recomendaciones
4. Tips prácticos de viaje

Formato: Claro, organizado, orientado a viajeros.
"""

    processing_response = await llm.ainvoke(processing_prompt)
    processed_research = processing_response.content

    print(colored("   ✅ Investigación completada", "green", attrs=["bold"]))

    # ========================================================================
    # PASO 7: Almacenar en el estado
    # ========================================================================
    return {
        "destination_research": processed_research,
        "destination": destination
    }

# ============================================================================
# NODO 4: GENERADOR DE ITINERARIO
# ============================================================================

async def study_guide_node(state: TravelState):
    """
    Nodo 4: Genera el itinerario de viaje personalizado

    Responsabilidades:
    - Crear itinerario día por día
    - Personalizar según preferencias y presupuesto
    - Incluir horarios, lugares específicos y tips
    """
    print(colored("\n📋 [GENERADOR DE ITINERARIO ACTIVADO]", "green", attrs=["bold"]))

    messages = state["messages"]
    destination = state.get("destination", "no especificado")
    research = state.get("destination_research", "")

    print(colored(f"   📊 Destino: {destination}", "yellow"))
    print(colored(f"   📝 Conversación: {len(messages)} mensajes", "yellow"))

    # Construir conversación completa
    full_conversation = "\n".join([
        f"{'Usuario' if isinstance(msg, HumanMessage) else 'Asistente'}: {msg.content}"
        for msg in messages
    ])

    # Prompt para generar itinerario
    itinerary_prompt = f"""
Eres un experto planificador de viajes. Crea un itinerario detallado y personalizado.

INFORMACIÓN DEL DESTINO:
{research}

CONVERSACIÓN COMPLETA:
{full_conversation}

INSTRUCCIONES:
Genera un itinerario día por día con:
- Título temático para cada día
- División en: Mañana / Tarde / Noche
- Horarios sugeridos
- Nombres específicos de lugares
- Precios aproximados según presupuesto
- Tips prácticos y recomendaciones
- Emojis para mejor visualización

Personaliza según:
- Duración mencionada
- Tipo de actividades preferidas
- Nivel de presupuesto
- Información del clima actual

Formato markdown, profesional pero amigable.
"""

    print(colored(f"   ⚙️  Generando itinerario para {destination}...", "yellow"))

    response = await llm.ainvoke(itinerary_prompt)

    print(colored("   ✅ Itinerario generado exitosamente", "green", attrs=["bold"]))

    return {
        "travel_itinerary": response.content
    }

print(colored("\n✅ Todos los nodos implementados correctamente", "green", attrs=["bold"]))



✅ Todos los nodos implementados correctamente


In [48]:
# CELDA 7: Función de Routing
from typing import Literal

def supervisor_router(state: TravelState) -> Literal["CONTINUE", "INVESTIGATE"]:
    """
    Router que decide el siguiente paso basado en la decisión del supervisor.

    Input: Estado actual
    Output: String que indica el camino a seguir

    Opciones:
    - "CONTINUE": Volver al entrevistador (falta información)
    - "INVESTIGATE": Ir a buscar info del destino (entrevista completa)
    """

    # Obtener la decisión del supervisor
    decision = state.get("supervisor_decision", "CONTINUE")

    print(colored(f"\n🎯 [ROUTER ACTIVADO]", "blue"))
    print(colored(f"   Decisión del supervisor: {decision}", "blue"))

    # Routing condicional
    if decision == "INVESTIGATE":
        print(colored(f"   → Ruta: Ir a INVESTIGADOR 🗺️", "green"))
        return "INVESTIGATE"
    else:
        print(colored(f"   → Ruta: Volver a ENTREVISTADOR 👤", "yellow"))
        return "CONTINUE"


print(colored("✅ Función de routing implementada", "green"))
print(colored("🔀 Decisiones posibles:", "cyan"))
print(colored("   - CONTINUE → Volver al entrevistador", "white"))
print(colored("   - INVESTIGATE → Ir al investigador", "white"))

✅ Función de routing implementada
🔀 Decisiones posibles:
   - CONTINUE → Volver al entrevistador
   - INVESTIGATE → Ir al investigador


In [49]:
# CELDA 8: Construcción del Grafo (con visualización)
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver

def create_travel_system():
    """
    Construye el sistema multi-agente completo.
    """

    print(colored("\n🏗️  [CONSTRUYENDO GRAFO]", "cyan", attrs=["bold"]))

    # ============================================================
    # PASO 1: Crear el grafo con el tipo de estado
    # ============================================================
    workflow = StateGraph(TravelState)
    print(colored("   ✓ Grafo inicializado con TravelState", "green"))

    # ============================================================
    # PASO 2: Agregar los 4 nodos (agentes)
    # ============================================================
    workflow.add_node("interviewer", interviewer_node)
    workflow.add_node("supervisor", supervisor_node)
    workflow.add_node("wikipedia", wikipedia_node)
    workflow.add_node("study_guide", study_guide_node)
    print(colored("   ✓ 4 nodos agregados al grafo", "green"))

    # ============================================================
    # PASO 3: Definir el punto de entrada
    # ============================================================
    workflow.add_edge(START, "interviewer")
    print(colored("   ✓ Punto de entrada: START → interviewer", "green"))

    # ============================================================
    # PASO 4: Flujo lineal simple (determinístico)
    # ============================================================
    workflow.add_edge("interviewer", "supervisor")
    print(colored("   ✓ Arista: interviewer → supervisor", "green"))

    # ============================================================
    # PASO 5: Decisión condicional del supervisor (CLAVE)
    # ============================================================
    workflow.add_conditional_edges(
        "supervisor",
        supervisor_router,
        {
            "CONTINUE": "interviewer",
            "INVESTIGATE": "wikipedia"
        }
    )
    print(colored("   ✓ Routing condicional configurado:", "green"))
    print(colored("     - CONTINUE → interviewer (loop)", "yellow"))
    print(colored("     - INVESTIGATE → wikipedia (avanzar)", "yellow"))

    # ============================================================
    # PASO 6: Flujo lineal después de Wikipedia
    # ============================================================
    workflow.add_edge("wikipedia", "study_guide")
    workflow.add_edge("study_guide", END)
    print(colored("   ✓ Aristas finales: wikipedia → study_guide → END", "green"))

    # ============================================================
    # PASO 7: Configurar memoria persistente (checkpointing)
    # ============================================================
    memory = MemorySaver()
    print(colored("   ✓ Memoria persistente configurada", "green"))

    # ============================================================
    # PASO 8: Compilar el grafo
    # ============================================================
    compiled_graph = workflow.compile(checkpointer=memory)
    print(colored("\n✅ GRAFO COMPILADO EXITOSAMENTE", "green", attrs=["bold"]))

    return compiled_graph


# Crear el sistema
travel_graph = create_travel_system()

print(colored("\n" + "="*60, "cyan"))
print(colored("🎉 SISTEMA MULTI-AGENTE LISTO PARA USAR", "cyan", attrs=["bold"]))
print(colored("="*60, "cyan"))

# ============================================================
# VISUALIZACIÓN ASCII DEL GRAFO (NUEVO)
# ============================================================
print(colored("\n📊 ARQUITECTURA DEL SISTEMA:", "yellow", attrs=["bold"]))
print(colored("""
        ┌─────────┐
        │  START  │
        └────┬────┘
             ↓
      ┌─────────────┐
      │ interviewer │ ←──────┐
      └──────┬──────┘        │
             ↓                │
      ┌─────────────┐         │
      │ supervisor  │         │
      └──────┬──────┘         │
             ↓                │
        [ROUTER]              │
             ↓                │
       ¿CONTINUE? ────────────┘
             │
       ¿INVESTIGATE?
             ↓
      ┌─────────────┐
      │  wikipedia  │
      └──────┬──────┘
             ↓
      ┌─────────────┐
      │ study_guide │
      └──────┬──────┘
             ↓
        ┌─────────┐
        │   END   │
        └─────────┘
""", "cyan"))

print(colored("🔄 Nodos: 4", "white"))
print(colored("   • interviewer (Entrevistador)", "white"))
print(colored("   • supervisor (Controlador)", "white"))
print(colored("   • wikipedia (Investigador)", "white"))
print(colored("   • study_guide (Generador)", "white"))

print(colored("\n🔀 Conexiones: 6", "white"))
print(colored("   • START → interviewer", "white"))
print(colored("   • interviewer → supervisor", "white"))
print(colored("   • supervisor → interviewer (si CONTINUE)", "white"))
print(colored("   • supervisor → wikipedia (si INVESTIGATE)", "white"))
print(colored("   • wikipedia → study_guide", "white"))
print(colored("   • study_guide → END", "white"))

print(colored("\n" + "="*60 + "\n", "cyan"))


🏗️  [CONSTRUYENDO GRAFO]
   ✓ Grafo inicializado con TravelState
   ✓ 4 nodos agregados al grafo
   ✓ Punto de entrada: START → interviewer
   ✓ Arista: interviewer → supervisor
   ✓ Routing condicional configurado:
     - CONTINUE → interviewer (loop)
     - INVESTIGATE → wikipedia (avanzar)
   ✓ Aristas finales: wikipedia → study_guide → END
   ✓ Memoria persistente configurada

✅ GRAFO COMPILADO EXITOSAMENTE

🎉 SISTEMA MULTI-AGENTE LISTO PARA USAR

📊 ARQUITECTURA DEL SISTEMA:

        ┌─────────┐
        │  START  │
        └────┬────┘
             ↓
      ┌─────────────┐
      │ interviewer │ ←──────┐
      └──────┬──────┘        │
             ↓                │
      ┌─────────────┐         │
      │ supervisor  │         │
      └──────┬──────┘         │
             ↓                │
        [ROUTER]              │
             ↓                │
       ¿CONTINUE? ────────────┘
             │
       ¿INVESTIGATE?
             ↓
      ┌─────────────┐
      │  wikipedia  │
    

In [53]:
# ============================================================================
# CELDA 9: PLAYGROUND INTERACTIVO (VERSIÓN CORREGIDA)
# ============================================================================

import uuid
from langchain_core.messages import HumanMessage

async def run_travel_planner():
    """
    Ejecuta el sistema de planificación de viajes de forma interactiva.
    """

    print(colored("""
╔══════════════════════════════════════════════════════════╗
║                                                          ║
║    ✈️  PLANIFICADOR DE VIAJES MULTI-AGENTE ✈️           ║
║                                                          ║
║    Sistema: LangGraph                                    ║
║    Agentes: 4 especializados                             ║
║    Modo: Interactivo                                     ║
║                                                          ║
╚══════════════════════════════════════════════════════════╝
    """, "cyan", attrs=["bold"]))

    print(colored("💡 Tip: El sistema te hará preguntas para planificar tu viaje", "yellow"))
    print(colored("💡 Responde naturalmente, como si hablaras con un agente de viajes\n", "yellow"))

    # ============================================================
    # CONFIGURACIÓN DE SESIÓN
    # ============================================================
    thread_id = f"travel_session_{uuid.uuid4().hex[:8]}"
    config = {
        "configurable": {"thread_id": thread_id},
        "recursion_limit": 50
    }

    print(colored(f"🔑 Sesión iniciada: {thread_id}\n", "cyan"))

    # ============================================================
    # ESTADO INICIAL
    # ============================================================
    current_state = {
        "messages": [],
        "destination": "",
        "duration": "",
        "activities": "",
        "budget": "",
        "supervisor_decision": "CONTINUE",
        "interview_complete": False,
        "destination_research": "",
        "travel_itinerary": ""
    }

    # ============================================================
    # PRIMERA EJECUCIÓN: Obtener saludo inicial
    # ============================================================
    print(colored("🔄 Iniciando conversación...\n", "cyan"))

    # Ejecutar una vez para obtener el saludo
    async for event in travel_graph.astream(current_state, config):
        for node_name, node_output in event.items():
            if node_name == "interviewer":
                current_state = node_output
                break
        break  # Solo queremos el primer mensaje

    # ============================================================
    # LOOP PRINCIPAL DE CONVERSACIÓN
    # ============================================================
    while True:

        # Mostrar el último mensaje del asistente
        if current_state.get("messages"):
            last_message = current_state["messages"][-1]
            if hasattr(last_message, 'content') and hasattr(last_message, 'type'):
                if last_message.type == 'ai':
                    print(colored(f"🤖 Asistente: {last_message.content}\n", "cyan"))

                    # ⭐ NUEVO: Detectar si la entrevista está completa en el mensaje
                    completion_signals = [
                        "entrevista_completa",
                        "¡entrevista_completa!",
                        "tengo todo lo necesario",
                        "listo para planificar"
                    ]

                    if any(signal in last_message.content.lower() for signal in completion_signals):
                        print(colored("\n⏳ Entrevista completada, generando itinerario...\n", "yellow", attrs=["bold"]))
                        # Marcar como completo
                        current_state["interview_complete"] = True

        # Verificar si ya terminamos
        if current_state.get("interview_complete", False):
            print(colored("\n" + "="*60, "magenta"))
            print(colored("✅ PLANIFICACIÓN COMPLETADA", "green", attrs=["bold"]))
            print(colored("="*60 + "\n", "magenta"))

            # Continuar ejecutando hasta el final (wikipedia + study_guide)
            async for event in travel_graph.astream(current_state, config):
                for node_name, node_output in event.items():
                    current_state = node_output

            # Mostrar resultados finales
            if current_state.get("destination_research"):
                print(colored("\n📚 INFORMACIÓN DEL DESTINO:", "cyan", attrs=["bold"]))
                print(colored("─" * 60, "cyan"))
                print(current_state["destination_research"])
                print(colored("─" * 60 + "\n", "cyan"))

            if current_state.get("travel_itinerary"):
                print(colored("\n🗓️  TU ITINERARIO PERSONALIZADO:", "green", attrs=["bold"]))
                print(colored("═" * 60, "green"))
                print(current_state["travel_itinerary"])
                print(colored("═" * 60 + "\n", "green"))

            print(colored("✈️  ¡Buen viaje! 🌍", "cyan", attrs=["bold"]))
            break

        # Pedir input al usuario
        print(colored("👤 Tú: ", "white", attrs=["bold"]), end="")
        user_input = input().strip()

        # Comando para salir
        if user_input.lower() in ["salir", "exit", "quit"]:
            print(colored("\n👋 ¡Hasta luego!", "yellow"))
            break

        # Agregar mensaje del usuario
        current_state["messages"].append(HumanMessage(content=user_input))

        print(colored("\n" + "─"*60, "white"))

        # Ejecutar el grafo con el nuevo mensaje
        async for event in travel_graph.astream(current_state, config):
            for node_name, node_output in event.items():
                current_state = node_output

                # Si llegamos al interviewer de nuevo, parar aquí para pedir input
                if node_name == "interviewer" and not current_state.get("interview_complete"):
                    break

            # Salir del stream si llegamos al interviewer
            if node_name == "interviewer" and not current_state.get("interview_complete"):
                break


# ============================================================
# EJECUTAR EL SISTEMA
# ============================================================
print(colored("\n🚀 Iniciando sistema...\n", "green", attrs=["bold"]))

# Ejecutar en modo asíncrono
await run_travel_planner()


🚀 Iniciando sistema...


╔══════════════════════════════════════════════════════════╗
║                                                          ║
║    ✈️  PLANIFICADOR DE VIAJES MULTI-AGENTE ✈️           ║
║                                                          ║
║    Sistema: LangGraph                                    ║
║    Agentes: 4 especializados                             ║
║    Modo: Interactivo                                     ║
║                                                          ║
╚══════════════════════════════════════════════════════════╝
    
💡 Tip: El sistema te hará preguntas para planificar tu viaje
💡 Responde naturalmente, como si hablaras con un agente de viajes

🔑 Sesión iniciada: travel_session_9cc0bb5c

🔄 Iniciando conversación...


👤 [ENTREVISTADOR ACTIVADO]
🤖 Asistente: ¡Hola! 👋 Soy tu asistente personal de viajes. ¿A qué destino te gustaría viajar? 🌍✈️

👤 Tú: Madrid

────────────────────────────────────────────────────────────

👤 [ENTREVISTADOR 